<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/wikipron-modeling/blob/master/wikipron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Installing Dependencies

In [ ]:
!pip install wikipron

In [ ]:
!pip install pynini ngram openfst

In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./

In [4]:
%cd ..

/content


# Preparing Data

In [ ]:
!git clone https://github.com/deterministic-algorithms-lab/wikipron-modeling

In [6]:
lg = 'hin'

In [ ]:
%cd wikipron-modeling/
!mkdir data

In [ ]:
%cd data
!mkdir {train,test,dev}

In [11]:
#!wikipron {lg} > {lg}.tsv
!shuf {lg}.tsv > shuf_{lg}.tsv
!rm {lg}.tsv
!mv shuf_{lg}.tsv {lg}.tsv

In [ ]:
#Cleans Data & splits it(80:10:10). [Remove short words + NFC normalisation + resolve multiple gold targets for same word]
!bash ../scripts/prepare

In [13]:
!mv {lg}_dev.tsv ./dev/
!mv {lg}_train.tsv ./train/
!mv {lg}_test.tsv ./test/

# Training

In [14]:
%cd ../nn

/content/wikipron-modeling/nn


In [15]:
!bash ./preprocess

In [ ]:
!fairseq-preprocess --source-lang {lg}.graphemes --target-lang {lg}.phonemes --trainpref ./data-txt/train \
--testpref ./data-txt/test --validpref ./data-txt/dev --destdir ./data-bin/{lg}/

In [ ]:
!bash ./sweep

# Evaluating On Samples

In [ ]:
%cd wikipron-modeling/nn

In [55]:
!mkdir evaluate

In [56]:
path_to_input_file = './data-txt/test.hin.graphemes'

In [57]:
tar_loc = './evaluate/train.'+lg+'.graphemes'
!mv {path_to_input_file} {tar_loc}

In [ ]:
src_dict = './data-bin/'+lg+'/dict.'+lg+'.graphemes.txt'
tgt_dict = './data-bin/'+lg+'/dict.'+lg+'.phonemes.txt'

!fairseq-preprocess --source-lang {lg}.graphemes --target-lang {lg}.phonemes  --only-source \
                    --trainpref './evaluate/train' --destdir './evaluate/data-bin/' \
                    --srcdict {src_dict} --tgtdict {tgt_dict}

In [61]:
!cp {tgt_dict} './evaluate/data-bin/'

In [62]:
import os
data_dir = './evaluate/data-bin/'
for f in os.listdir(data_dir):
    path = os.path.join(data_dir,f)
    if os.path.isfile(path) and f.startswith('train'):
        new_path = os.path.join(data_dir, 'test'+f[5:])
        os.rename(path, new_path)

In [ ]:
checkpoint_path = 'checkpoints/'+lg+'-64-128-64-128/'+'checkpoint_last.pt'

!fairseq-generate './evaluate/data-bin/' \
    --path {checkpoint_path}  --source-lang {lg}.graphemes \
    --target-lang {lg}.phonemes --batch-size 1 --beam 5